<a href="https://colab.research.google.com/github/a-forty-two/cylons/blob/master/08_SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# NLTK, Spacy, Textacy -> MANIPULATE TEXT -> NLP Pipelines 

# ML -> NUMBERS only 
# Approaches to CONVERT text into NUMBERS or VECTORS or TENSORS 
# y = mx + c where x is an language statement 

# MANIPULATED standards -> CONVINIENCE 

# SPECIAL SYMBOLS INSIDE OUR TEXT so that code could be used to manipulate them 
# WE AS DEVELOPERS OR NLP Engineers manipulate text and add these symbols
# <PAD>   0   -> Neural networks are HARDCODED-> input-size cannot be change once NN is built!
# padding helps chopping/padding different length sentences into same size
# I am good;   Bye.   ->    [I, am, good] ; [Bye, <PAD>, <PAD>]
# y = mx+c ;   y = w1*I + w2*am + w3*good + bias ;   y = w1*bye + w2 * <PAD> + w3 * <PAD>
# y = w1*100 + w2*22 + w3*55 + 102;   y = w1*42 + w2* 0 + w3* 0  
# <START> 1   -> indicates beginning of a sentence or phrase 
# example; if text was Hello World! -> 42 50 
# Processed statement: <START> Hello World! -> 1 42 50
# this way when multiple sentences were present, we could mark beginning of each sentence/phrase

# <PAD> 0          -> 0, any weight multiplied to <PAD> will become 0! 
# <START>  1       -> Indicates beginning of sentence/phrase 
# <UNK>  2         -> your encountered a word outside dictionary 
# <UNUSED> 3       -> SPECIAL symbol that you could give special meaning to 




In [5]:
# IMDB -> has a lot of movie reviews 
# POS, NEG -> our output for a Review 
# Sentiment = weights * review + bias 

%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)
from tensorflow import keras
import numpy as np

1.15.2


In [2]:

# DICTIONARIES are infinite! OXFORD, CAMBRIDGE, QUEEN's, GEORGE's, millions of words!
# constantly increasing!
# WE HAVE TO LIMIT OUR DICTIONARY TO A FINITE SIZE! 
 # assuming our entire english consists of only 10,000 words! 
# REST of the words -> <UNKNOWN>!!!!
imdb = keras.datasets.imdb 
HP_dictionary_size = 10000
(xtrain, ytrain), (xtest, ytest) = imdb.load_data(num_words=HP_dictionary_size)



17465344/17464789 [==============================] - 0s 0us/step


In [9]:
print(xtrain[2])

[1, 14, 47, 8, 30, 31, 7, 4, 249, 108, 7, 4, 5974, 54, 61, 369, 13, 71, 149, 14, 22, 112, 4, 2401, 311, 12, 16, 3711, 33, 75, 43, 1829, 296, 4, 86, 320, 35, 534, 19, 263, 4821, 1301, 4, 1873, 33, 89, 78, 12, 66, 16, 4, 360, 7, 4, 58, 316, 334, 11, 4, 1716, 43, 645, 662, 8, 257, 85, 1200, 42, 1228, 2578, 83, 68, 3912, 15, 36, 165, 1539, 278, 36, 69, 2, 780, 8, 106, 14, 6905, 1338, 18, 6, 22, 12, 215, 28, 610, 40, 6, 87, 326, 23, 2300, 21, 23, 22, 12, 272, 40, 57, 31, 11, 4, 22, 47, 6, 2307, 51, 9, 170, 23, 595, 116, 595, 1352, 13, 191, 79, 638, 89, 2, 14, 9, 8, 106, 607, 624, 35, 534, 6, 227, 7, 129, 113]


In [7]:
ytrain[:5]
# there are 2 outputs -> 0 or 1

array([1, 0, 0, 1, 0])

In [13]:
word_index = imdb.get_word_index()   # DICTIONARY HAS TO BE PROVIDED BY DATASET!
# WHOEVER BUILDS DATASET HAS TO BUILD DICTIONARY AS PER THEIR ENCODING 
word_index['bye']
# PRECISELY OPPOSITE of what we need! This can help us with ENCODING, not decoding
# ENCODING is already done! 
# how do we decode? -> REVERSE THE DICTIONARY!!
# Pass it word -> GIVES you a number!
# Our reverse dict -> PASS a number, and get word back 

5455